# 00. lab 00 - obliczenia


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- 1. DANE POMIAROWE (Z raportu) ---

# Znane pojemności [uF]
C_known = np.array([1, 2, 3, 4.5, 5, 6.5, 7, 8, 9])

# Czasy t20 [mm:ss,ms] dla Serii I (R = 1.4 MOhm)
# Przepisane z Table 1 w raporcie
times_R1_raw = [
    "0:12,63", "0:25,62", "0:37,75", "0:58,0", "1:04,00", 
    "1:23,84", "1:31,84", "1:44,03", "1:57,28"
]

# Czasy t20 [mm:ss,ms] dla Serii II (R = 2.6 MOhm)
# Przepisane z Table 2 w raporcie
times_R2_raw = [
    "0:23,32", "0:46,37", "1:09,15", "1:43,84", "1:50,07",
    "2:30,91", "2:43,43", "3:07,75", "3:30,66"
]

# Czasy t20 dla nieznanego Cx
time_Cx_R1_raw = "0:45,65"
time_Cx_R2_raw = "1:29,22"

# Stałe i niepewności przyrządów
R1 = 1.4e6      # 1.4 MOhm
R2 = 2.6e6      # 2.6 MOhm
N_cycles = 20   # liczba cykli
u_T_inst = 0.01 # niepewność stopera [s] (założona w raporcie)
u_C_inst = 0.05e-6 # niepewność kondensatora [F] (założona w raporcie)

# --- 2. FUNKCJE POMOCNICZE ---

def parse_time(t_str):
    """Konwertuje czas z formatu 'mm:ss,ms' na sekundy (float)."""
    # Zamiana przecinka na kropkę
    t_str = t_str.replace(',', '.')
    
    if ':' in t_str:
        parts = t_str.split(':')
        minutes = float(parts[0])
        seconds = float(parts[1])
        return minutes * 60 + seconds
    else:
        return float(t_str)

def calc_uncertainty_K(T, R, C, K_val):
    """Oblicza u(K) metodą różniczki zupełnej (propagacja błędu)."""
    # Wzór: u(K) = sqrt( (1/RC * uT)^2 + (-T/(RC^2) * uC)^2 )
    term_T = (1 / (R * C)) * u_T_inst
    term_C = (-T / (R * C**2)) * u_C_inst
    return np.sqrt(term_T**2 + term_C**2)

def calc_uncertainty_Cx(Tx, R, K, Cx_val, u_K_val):
    """Oblicza u(Cx) metodą różniczki zupełnej."""
    # Wzór: u(Cx) = sqrt( (1/KR * uTx)^2 + (-Tx/(K^2 R) * uK)^2 )
    term_Tx = (1 / (K * R)) * u_T_inst
    term_K = (-Tx / (K**2 * R)) * u_K_val
    return np.sqrt(term_Tx**2 + term_K**2)

# --- 3. OBLICZENIA GŁÓWNE ---

# Konwersja pojemności na Farady
C_F = C_known * 1e-6

# --- Seria I (R = 1.4 M) ---
times_R1 = np.array([parse_time(t) for t in times_R1_raw])
T_R1 = times_R1 / N_cycles
# Obliczenie K = T / (R * C)
K_R1 = T_R1 / (R1 * C_F)

# --- Seria II (R = 2.6 M) ---
times_R2 = np.array([parse_time(t) for t in times_R2_raw])
T_R2 = times_R2 / N_cycles
K_R2 = T_R2 / (R2 * C_F)

# Średnie wartości K
K_mean_R1 = np.mean(K_R1)
K_mean_R2 = np.mean(K_R2)
K_mean_total = (K_mean_R1 + K_mean_R2) / 2

# --- Nieznana pojemność Cx ---
# Seria I
t_Cx_R1 = parse_time(time_Cx_R1_raw)
T_Cx_R1 = t_Cx_R1 / N_cycles
Cx_R1 = T_Cx_R1 / (K_mean_total * R1)

# Seria II
t_Cx_R2 = parse_time(time_Cx_R2_raw)
T_Cx_R2 = t_Cx_R2 / N_cycles
Cx_R2 = T_Cx_R2 / (K_mean_total * R2)

# Średnia Cx
Cx_mean = (Cx_R1 + Cx_R2) / 2

# --- 4. NIEPEWNOŚCI (Odtworzenie przykładów z raportu) ---

# Przykład 1: u(K) dla Serii I, C=1uF
# W raporcie u(K) wyszło ok 0.024
u_K_ex1 = calc_uncertainty_K(T_R1[0], R1, C_F[0], K_R1[0])

# Przyjmujemy u(K) do dalszych obliczeń jako średnią lub wartość z przykładu 
# (Twój zespół użył wartości ~0.024 w przykładach dla Cx)
u_K_for_calc = 0.024 

# Przykład 2: u(Cx) dla R=1.4M
u_Cx_R1 = calc_uncertainty_Cx(T_Cx_R1, R1, K_mean_total, Cx_R1, u_K_for_calc)

# Przykład 3: u(Cx) dla R=2.6M
u_Cx_R2 = calc_uncertainty_Cx(T_Cx_R2, R2, K_mean_total, Cx_R2, u_K_for_calc)

# Niepewność średniej Cx: 1/2 * sqrt(u1^2 + u2^2)
u_Cx_mean = 0.5 * np.sqrt(u_Cx_R1**2 + u_Cx_R2**2)

# --- 5. WYPISANIE WYNIKÓW (Formatowanie pod raport) ---

print("="*60)
print("     WYNIKI DO SPRAWOZDANIA (Exercise 54)")
print("="*60)

print(f"\n--- WYZNACZANIE STAŁEJ K ---")
print(f"K_sr (Seria I):  {K_mean_R1:.4f}")
print(f"K_sr (Seria II): {K_mean_R2:.4f}")
print(f"K_sr (Całość):   {K_mean_total:.4f}  <-- Do użycia w obliczeniach Cx")

print(f"\n--- WYZNACZANIE Cx ---")
print(f"Cx (z R=1.4M): {Cx_R1 * 1e6:.4f} uF")
print(f"Cx (z R=2.6M): {Cx_R2 * 1e6:.4f} uF")
print(f"Cx (Średnia):  {Cx_mean * 1e6:.4f} uF")

print(f"\n--- NIEPEWNOŚCI (Weryfikacja przykładów) ---")
print(f"u(K) przykład (C=1uF, R=1.4M): {u_K_ex1:.4f}")
print(f"u(Cx) przykład (R=1.4M):       {u_Cx_R1 * 1e6:.4f} uF")
print(f"u(Cx) przykład (R=2.6M):       {u_Cx_R2 * 1e6:.4f} uF")
print(f"u(Cx_mean) końcowe:            {u_Cx_mean * 1e6:.4f} uF")

print(f"\n[WYNIK KOŃCOWY]: Cx = ({Cx_mean * 1e6:.2f} +/- {u_Cx_mean * 1e6:.2f}) uF")

print("\n" + "="*60)
print("     DANE DO TABEL (Kopiuj do LaTeX)")
print("="*60)

print("\n--- TABELA WYNIKÓW: Seria I (R=1.4 M) ---")
print(f"{'C [uF]':<8} | {'t20 [s]':<10} | {'T [s]':<10} | {'K':<10}")
print("-" * 46)
for i in range(len(C_known)):
    print(f"{C_known[i]:<8.1f} | {times_R1[i]:<10.2f} | {T_R1[i]:<10.4f} | {K_R1[i]:<10.4f}")

print("\n--- TABELA WYNIKÓW: Seria II (R=2.6 M) ---")
print(f"{'C [uF]':<8} | {'t20 [s]':<10} | {'T [s]':<10} | {'K':<10}")
print("-" * 46)
for i in range(len(C_known)):
    print(f"{C_known[i]:<8.1f} | {times_R2[i]:<10.2f} | {T_R2[i]:<10.4f} | {K_R2[i]:<10.4f}")

# --- 6. RYSOWANIE WYKRESU ---
plt.figure(figsize=(10, 6))

# Seria 1
plt.plot(C_known, T_R1, 'bo', label=r'Pomiary R=1.4 M$\Omega$')
# Dopasowanie liniowe (regresja) dla wizualizacji
a1, b1 = np.polyfit(C_known, T_R1, 1)
plt.plot(C_known, a1*C_known + b1, 'b--', alpha=0.5, label=f'Regresja R1')

# Seria 2
plt.plot(C_known, T_R2, 'rs', label=r'Pomiary R=2.6 M$\Omega$')
a2, b2 = np.polyfit(C_known, T_R2, 1)
plt.plot(C_known, a2*C_known + b2, 'r--', alpha=0.5, label=f'Regresja R2')

# Zaznaczenie punktów Cx (obliczonych) na wykresie
# T_Cx to średni czas cyklu dla nieznanego kondensatora
plt.scatter([Cx_R1*1e6], [T_Cx_R1], color='cyan', zorder=5, edgecolors='black', s=100,
            label=fr'Cx1 (obl)={Cx_R1*1e6:.2f}$\mu$F')
plt.scatter([Cx_R2*1e6], [T_Cx_R2], color='magenta', zorder=5, edgecolors='black', s=100,
            label=fr'Cx2 (obl)={Cx_R2*1e6:.2f}$\mu$F')

plt.title(r"Zależność okresu drgań relaksacyjnych od pojemności $T = f(C)$")
plt.xlabel(r"Pojemność C [$\mu$F]")
plt.ylabel("Okres T [s]")
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend()
plt.tight_layout()

# Zapisanie wykresu
output_filename = "wykres_TC_poprawiony.png"
plt.savefig(output_filename, dpi=300)
print(f"\n[INFO] Wykres zapisano jako '{output_filename}'")
plt.show()